In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import quandl

In [2]:
quandl_key = os.getenv("QUANDL_API_KEY")
mydata=quandl.get("USTREASURY/YIELD", authtoken=quandl_key, start_date="1900-12-31", end_date="2022-12-31")
mydata.head()


ten_years = mydata[['10 YR']]
ten_years.tail()


,10 YR
Date,
2021-03-10,1.53
2021-03-11,1.54
2021-03-12,1.64
2021-03-15,1.62
2021-03-16,1.62


In [3]:
load_dotenv()
# Set Alpaca API key and secret
alpaca_key = os.getenv('ALPACA_API_KEY')
alpaca_secret = os.getenv('ALPACA_SECRET_KEY')

api = tradeapi.REST(
    alpaca_key,
    alpaca_secret,
    api_version = "v2"
)

In [4]:
tickers = ["XLY", "XLP", "XLE","XLF", "XLV", "XLI", "XLB", "XLK", "XLU"]

timeframe = "1D"
year_start=1990
year_list=[]
while year_start < 2022:
    year_list.append(year_start)
    year_start +=1

ticker_data = []
for my_year in year_list:
    
    start_date=str(my_year)+'-01-01'
    end_date=str(my_year)+'-12-31'

    data = api.get_barset(
    tickers,
    timeframe,
    start = pd.Timestamp(start_date, tz="America/New_York").isoformat(),
    end = pd.Timestamp(end_date, tz="America/New_York").isoformat(),
    limit = 1000
    ).df
    ticker_data.append(data)
    
all_ticker_data = pd.concat(ticker_data, axis="rows", join="inner")

all_ticker_data.index.rename("Date", inplace=True)
all_ticker_data.index = pd.to_datetime(all_ticker_data.index.date)


all_ticker_data.head()


XLB                                   XLE                       \
             open   high    low  close    volume   open   high    low  close   
2008-01-02  41.70  41.95  40.98  41.33   9483635  79.95  80.30  79.27  79.75   
2008-01-03  41.52  42.17  41.45  41.80   8699808  79.94  80.75  79.85  80.21   
2008-01-04  41.42  41.57  40.70  40.74  13370213  79.50  79.74  77.88  78.01   
2008-01-07  40.81  41.06  39.67  40.25  16971407  78.00  78.61  76.23  77.04   
2008-01-08  40.40  41.07  39.55  39.60  21112177  77.88  78.19  75.85  76.08   

                      ...    XLV                                  XLY         \
              volume  ...   open   high    low  close   volume   open   high   
2008-01-02  18291924  ...  35.63  35.63  34.86  34.94  3915069  32.99  32.99   
2008-01-03  14113740  ...  34.98  35.30  34.56  35.18  2504282  32.11  32.22   
2008-01-04  24552334  ...  35.01  35.16  34.50  34.83  3214308  31.43  31.58   
2008-01-07  29186338  ...  34.82  35.58  34.82  35.57  5451740  31.14  31.18   
2008-01-08  20801316  ...  35.66  36.39  35.57  35.81  6210752  31.03  31.31   

                                     
              low  close     volume  
2008-01-02  32.11  32.13  3711800.0  
2008-01-03  31.68  31.78  9585427.0  
2008-01-04  30.75  30.78  5960834.0  
2008-01-07  30.68  30.93  6104205.0  
2008-01-08  30.31  30.32  4287009.0  

[5 rows x 45 columns]

In [5]:
closing_prices = pd.DataFrame()
for ticker in tickers:
    closing_prices[ticker]=all_ticker_data[ticker]["close"]

    
combine_dataframe = pd.concat([closing_prices ,ten_years], axis="columns", join="inner")
combine_dataframe.index.name = 'Date'


combine_dataframe.to_csv("historical_data.csv")

combine_dataframe.head()

,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,10 YR
Date,,,,,,,,,,
2008-01-02,32.13,28.34,79.75,28.41,34.94,38.37,41.33,26.10,41.98,3.91
2008-01-03,31.78,28.22,80.21,28.20,35.18,38.58,41.80,26.08,42.08,3.91
2008-01-04,30.78,28.04,78.01,27.39,34.83,37.60,40.74,25.07,42.30,3.88
2008-01-07,30.93,28.55,77.04,27.55,35.57,37.34,40.25,24.97,43.25,3.86
2008-01-08,30.32,28.38,76.08,26.54,35.81,36.39,39.60,24.21,43.33,3.86
